# Reading networks from SQL databases

[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/sql.ipynb)  
[Download notebook](https://github.com/pathpy/pathpy/raw/master/doc/tutorial/sql.ipynb)

To simplify the analysis of network data, `pathpy` can directly load and write network from and into SQL databases. For this, we can call the function `read_sql` function in the `io` module. It allows us to pass an open connection to an SQLite database file, which we obtain using the `connect` function of sqlite3. We can further pass an arbitrary SQL query. The columns of this query will be used to generate edges with attributes. 

In [8]:
pip install git+git://github.com/pathpy/pathpy.git

  Cloning git://github.com/pathpy/pathpy.git to /tmp/pip-req-build-jr37pdqd
  Running command git clone -q git://github.com/pathpy/pathpy.git /tmp/pip-req-build-jr37pdqd
You should consider upgrading via the '/home/max/py3-venv/pathp/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pathpy as pp
import sqlite3

In [2]:
con = sqlite3.connect('networks.db')
n = pp.io.sql.read_network(con=con, sql='SELECT source, target FROM lotr', directed=True)
print(n)

DatabaseError: Execution failed on sql 'SELECT source, target FROM lotr': no such table: lotr

If we instead want to read all columns from a given table (without giving an open connection or an SQL query) we can simply write:

In [4]:
n = pp.io.sql.read_network(filename='networks.db', table='lotr')
print(n)

[05-21 17:46:46: ERROR] Either an SQL connection or a filename is required


ParameterError: Either an SQL connection or a filename is required

If we read a whole table from an SQL database, the `read_sql` function will automatically retrieve all edge attributes. In our database, the table `lotr`, which captures proximity of Lord of the Rings characters as the story progresses, contains a time attribute that stores when (in which sentence) both characters are mentioned.

In [5]:
n = pp.io.read_sql(filename='networks.db', table='lotr', directed=True)
e = list(n.edges.uids)[0]
print(n.edges[e])
print(n.edges[e].attributes)

AttributeError: module 'pathpy.io' has no attribute 'read_sql'

toring a Network in an SQL database is just as easy. Let's first create a network where edges have attributes:

In [6]:
n = pp.Network(directed=False)
n.add_edge('a', 'b', weight=2.0)
n.add_edge('a', 'c', type='friendship')
print(n)

Uid:			0x7f59030963d0
Type:			Network
Directed:		False
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2


To store this in a new table in an SQLite database file, we call the `write_sql` function. If the database file and table do not exist, a new database file or table is created. if they exist, we can use the `if_exists` parameter to specify what should be done:

In [9]:
pp.io.sql.write(n, filename='networks.db', table='test_network', if_exists='replace')

In [11]:
con = sqlite3.connect('networks.db')
n = pp.io.sql.read_network(con=con, sql='SELECT * FROM test_network', directed=True)
print(n)

Uid:			0x7f59024530a0
Type:			Network
Directed:		True
Multi-Edges:		False
Number of nodes:	3
Number of edges:	2
